In [1]:
from database.database import DatabaseProcessor
from Embedders import get_embedder
from dotenv import load_dotenv
import os

load_dotenv('.env', override=True)

True

In [2]:
db_params = {
    'dbname': os.getenv('DB_NAME') ,
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT')
}
db = DatabaseProcessor(db_params)
db.test_connection()
embedder = get_embedder(model_name="adsabs/astroBERT",
                        device='mps', normalize=False)
print(embedder)

=============================CONFIG=============================
Database         User             Host             Port            
citeline_db      bbasseri         localhost        5432            
Database version: ('PostgreSQL 17.3 (Homebrew) on x86_64-apple-darwin23.6.0, compiled by Apple clang version 16.0.0 (clang-1600.0.26.6), 64-bit',)
adsabs/astroBERT, normalize=False


In [3]:
# Get 4096 chunks from the database
import psycopg2

def get_chunks(num: int):
    conn = psycopg2.connect(**db.db_params)
    cur = conn.cursor()
    cur.execute("SELECT text FROM chunks LIMIT %s", (num,))
    rows = cur.fetchall()
    cur.close()
    conn.close()
    return [row[0] for row in rows]

In [6]:
batch_size = 1

while batch_size < 4096:
    try:
        chunks = get_chunks(batch_size)
        print(f"Got {len(chunks)} chunks")
        embeddings = embedder(chunks)
        print(embeddings.shape)
        batch_size *= 2
        print(f"increasing batch size to {batch_size}")
    except Exception as e:
        print(e)

        break


Got 1 chunks
(1, 768)
increasing batch size to 2
Got 2 chunks
(2, 768)
increasing batch size to 4
Got 4 chunks
(4, 768)
increasing batch size to 8
Got 8 chunks
(8, 768)
increasing batch size to 16
Got 16 chunks
(16, 768)
increasing batch size to 32
Got 32 chunks
(32, 768)
increasing batch size to 64
Got 64 chunks
(64, 768)
increasing batch size to 128
Got 128 chunks
(128, 768)
increasing batch size to 256
Got 256 chunks


KeyboardInterrupt: 